# CXR inference TF-TRT

#### DenseNet201로 학습된 모델의 Inference 성능 향상을 위해 TensroRT를 사용해보자.

In [2]:
import tensorflow as tf
import numpy as np
import time
import os
from tensorflow.python.compiler.tensorrt import trt_convert as trt
from tensorflow.python.saved_model import tag_constants
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from Seg_modules import *
from PIL import Image
from tqdm import tqdm

Check TensorFlow version

In [1]:
tf.__version__

NameError: name 'tf' is not defined

Check GPU

In [ ]:
# !nvidia-smi

Check TensorRT version

In [4]:
print("TensorRT version: ")
!dpkg -l | grep nvinfer

TensorRT version: 
ii  libnvinfer-bin              7.2.1-1+cuda11.1                    amd64        TensorRT binaries
ii  libnvinfer-dev              7.2.1-1+cuda11.1                    amd64        TensorRT development libraries and headers
ii  libnvinfer-plugin-dev       7.2.1-1+cuda11.1                    amd64        TensorRT plugin libraries
ii  libnvinfer-plugin7          7.2.1-1+cuda11.1                    amd64        TensorRT plugin libraries
ii  libnvinfer7                 7.2.1-1+cuda11.1                    amd64        TensorRT runtime libraries
ii  python3-libnvinfer          7.2.1-1+cuda11.1                    amd64        Python 3 bindings for TensorRT
ii  python3-libnvinfer-dev      7.2.1-1+cuda11.1                    amd64        Python 3 development package for TensorRT


# Inference

## Native Inference Code

In [ ]:
# Load models
seg_model = load_model('../models/image_models/seg_model.h5',custom_objects={'dice_coef_loss': dice_coef_loss,'dice_coef':dice_coef})
DenseNet = load_model('../models/image_models/DenseNet201.h5')
img_model = load_model('../models/image_models/Covid_2class.h5')

In [ ]:
# image path

# COVID
image_path="./test_data/covid-1.png"

# NON-COVID
# image_path = "./test_data/normal-1.jpg"

In [ ]:
img_size=(224,224)
label_list = ('negative','positive')

start_time = time.time()

# LOAD image and data preprocessing
cropped_image = get_cropped_image(image_path, seg_model)

img = tf.keras.preprocessing.image.array_to_img(cropped_image)
img = img.resize(img_size)
img = tf.keras.preprocessing.image.img_to_array(img)
img = img / 255.0
img = np.expand_dims(img, axis=0)

feature_vector = DenseNet.predict(img)

prediction = img_model.predict(feature_vector)[0]

end_time = time.time()

elapsed_time = end_time - start_time
print("경과 시간: ", elapsed_time)

print("prediction: ", prediction)

val = prediction.item(0)
print("val: ", val)

idx = int(np.round(val))
print("idx: ", idx)

label = label_list[idx]
print("label: ", label)

## FP16 Inference Code

### TF-TRT FP16 model

In [ ]:
def make_FP16_model(input_saved_model_dir, output_saved_model_dir):
    print('Converting to TF-TRT FP16...')

    conversion_params = trt.DEFAULT_TRT_CONVERSION_PARAMS._replace(
        precision_mode=trt.TrtPrecisionMode.FP16,
        max_workspace_size_bytes=8000000000)

    converter = trt.TrtGraphConverterV2(
       input_saved_model_dir=input_saved_model_dir, conversion_params=conversion_params)

    converter.convert()

    converter.save(output_saved_model_dir=output_saved_model_dir)

    print('Done Converting to TF-TRT FP16')
    
    return True

In [ ]:
make_FP16_model("../models/image_models/DenseNet201", "./model/CXR_DenseNet_FP16_saved_model")

In [ ]:
make_FP16_model("../models/image_models/Covid_2class", "./model/CXR_C2C_FP16_saved_model")

In [ ]:
make_FP16_model("../models/image_models/seg_model", "./model/CXR_seg_FP16_saved_model")

## Load models

In [ ]:
seg_model = load_model('../models/image_models/seg_model.h5',custom_objects={'dice_coef_loss': dice_coef_loss,'dice_coef':dice_coef})
DenseNet = load_model('./model/CXR_DenseNet_FP16_saved_model')
img_model = load_model('./model/CXR_C2C_FP16_saved_model')

In [ ]:
# MAKE inference Function
def predict_cxr(image_path):
    img_size = (224, 224)
    label_list = ('negative','positive')
    cropped_image = get_cropped_image(image_path, seg_model)
    img = tf.keras.preprocessing.image.array_to_img(cropped_image)
    img = img.resize(img_size)
    img = tf.keras.preprocessing.image.img_to_array(img)
    img = img / 255.0
    img = np.expand_dims(img, axis=0)
    cropped_img = tf.constant(img)
        
    signature_keys = list(DenseNet.signatures.keys())
    infer = DenseNet.signatures[signature_keys[0]]
    
    pred = infer(cropped_img)

    key = list(pred.keys())[0]
    val = pred[key]
    
    signature_keys = list(img_model.signatures.keys())
    inference = img_model.signatures[signature_keys[0]]
    
    prediction = inference(val)
    
    key = list(prediction.keys())[0]
    value = prediction[key].numpy()[0]
    
    idx = int(np.round(value))
    label = label_list[idx]
        
    return label

In [ ]:
start_time = time.time()

infer = predict_cxr(image_path)

end_time = time.time()

print("경과 시간: ", end_time - start_time)
print("Inference: ", infer)